In [ ]:
import cv2
import argparse
import time
import os,sys
import eel
import glob
import random
!pip install import-ipynb
import import_ipynb
import Update_Model

eel.init('WD')
emotions=["angry", "happy", "sad", "neutral"]
fishface = cv2.face.FisherFaceRecognizer_create()
font = cv2.FONT_HERSHEY_SIMPLEX


parser=argparse.ArgumentParser(description="Options for emotions based music player(Updating the model)")
parser.add_argument("--update", help="Call for taking new images and retraining the model.", action="store_true")
args, unknown = parser.parse_known_args()  
facedict={}

video_capture=cv2.VideoCapture(0)
#cv2.namedWindow("Window")

facecascade=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
def crop(clahe_image, face):
    for (x, y, w, h) in face:
        faceslice=clahe_image[y:y+h, x:x+w]
        faceslice=cv2.resize(faceslice, (48, 48))
        facedict["face%s" %(len(facedict)+1)]=faceslice
    return faceslice

def grab_face(): 
    while True:
        ret, frame = video_capture.read()
        cv2.imshow("Window", frame) 
        if cv2.waitKey(1) & 0xFF == ord('q'): 
            break
        cv2.imwrite('test.jpg', frame)
        cv2.imwrite("images/main%s.png" %count, frame)
        gray=cv2.imread('test.jpg',0)
        gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        clahe=cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        clahe_image=clahe.apply(gray)
        return clahe_image
    cv2.waitKey(0)
    cv2.destroyAllWindows()
            
        
       

    
        

def detect_face():
    clahe_image=grab_face()
    face=facecascade.detectMultiScale(clahe_image, scaleFactor=1.1, minNeighbors=15, minSize=(10, 10), flags=cv2.CASCADE_SCALE_IMAGE)
    if len(face)>=1:
        faceslice=crop(clahe_image, face)
    else:
        print("No/Multiple faces detected!!, passing over the frame")
        

def save_face(emotion):
    print("\n\nLook "+emotion+" untill the timer expires and keep the same emotion for some time.")
    print('\a')
    for i in range(0, 5):
        print(5-i)
        time.sleep(1)
    while len(facedict.keys())<16:
        detect_face()
    for i in facedict.keys():
        path, dirs, files = next(os.walk(r"C:\Users\Dell\MOODIFY_SMART MUSIC PLAYER BASED ON FACIAL EMOTIONS\data/%s" %emotion))
        file_count = len(files)+1
        cv2.imwrite(r"C:\Users\Dell\MOODIFY_SMART MUSIC PLAYER BASED ON FACIAL EMOTIONS\data/%s/%s.jpg" %(emotion, (file_count)), facedict[i])
    facedict.clear()

def update_model(emotions):
    print("Update mode for model is ready")
    checkForFolders(emotions)
    
    for i in range(0, len(emotions)):
        save_face(emotions[i])
    print("Collected the images, looking nice! Now updating the model...")
    Update_Model.update(emotions)
    print("Model train successful!!")

def checkForFolders(emotions):
    for emotion in emotions:
        if os.path.exists(r"C:\Users\Dell\MOODIFY_SMART MUSIC PLAYER BASED ON FACIAL EMOTIONS\data/%s" %emotion):
            pass
        else:
            os.makedirs(r"C:\Users\Dell\MOODIFY_SMART MUSIC PLAYER BASED ON FACIAL EMOTIONS\data/%s" %emotion)

def identify_emotions():
    prediction=[]
    confidence=[]

    for i in facedict.keys():
        pred, conf=fishface.predict(facedict[i])
        cv2.imwrite("images/%s.jpg" %i, facedict[i])
        prediction.append(pred)
        confidence.append(conf)
    output=emotions[max(set(prediction), key=prediction.count)]    
    print("You are %s" %output) 
    facedict.clear()
    return output
    
count=0
@eel.expose
def getEmotion():
    count=0
    while True:
        count=count+1
        detect_face()
        if args.update:
            update_model(emotions)
            break
        elif count==10:
            fishface.read("model2.xml")
            return identify_emotions()
            break

eel.start('main.html')


You are neutral
No/Multiple faces detected!!, passing over the frame
No/Multiple faces detected!!, passing over the frame
No/Multiple faces detected!!, passing over the frame
No/Multiple faces detected!!, passing over the frame
No/Multiple faces detected!!, passing over the frame
No/Multiple faces detected!!, passing over the frame
No/Multiple faces detected!!, passing over the frame
No/Multiple faces detected!!, passing over the frame
No/Multiple faces detected!!, passing over the frame
You are angry
